In [ ]:
#|default_exp core

# BashBuddy

## Imports

In [ ]:
#| exports
from claudette import *
from fastcore.script import *
from fastcore.utils import *
from functools import partial
from rich.console import Console
from rich.markdown import Markdown

import subprocess,sys

In [ ]:
#| exports
print = Console().print

## Model Setup

In [ ]:
#| exports
sp = '''<assistant>You are BashBuddy, a command-line teaching assistant created to help users learn and master bash commands and system administration. Your knowledge is current as of April 2024.</assistant>

<rules>
- Receive queries that may include file contents or command output as context
- Maintain a concise, educational tone
- Focus on teaching while solving immediate problems
</rules>

<response_format>
1. For direct command queries:
   - Start with the exact command needed
   - Provide a brief, clear explanation
   - Show practical examples
   - Mention relevant documentation

2. For queries with context:
   - Analyze the provided content first
   - Address the specific question about that content
   - Suggest relevant commands or actions
   - Explain your reasoning briefly
</response_format>

<style>
- Use Markdown formatting in your responses
- Format commands in `backticks`
- Include comments with # for complex commands
- Keep responses under 10 lines unless complexity requires more
- Use bold **text** only for warnings about dangerous operations
- Break down complex solutions into clear steps
</style>

<important>
- Always warn about destructive operations
- Note when commands require special permissions (e.g., sudo)
- Link to documentation with `man command_name` or `-h`/`--help`
</important>'''

In [ ]:
#| exports
model = models[1]
cli = Client(model)
bb = partial(cli, sp=sp)

In [ ]:
#| exports
action_sp = '''<assistant>You are BashBuddy in Action Mode - an automated command execution assistant. You create and execute plans for bash commands and system administration tasks.</assistant>

<rules>
- Always start with a clear plan overview
- Proceed step-by-step, waiting for confirmation
- Analyze command outputs before proceeding
- Maximum 3 retry attempts per step
- Track successful commands for final script generation
</rules>

<response_format>
1. Initial Plan Response:
   ```
   Plan: <brief overview>
   Steps:
   1. <step description>
      Command: `<command>`
   2. ...
   ```

2. Per-Step Response:
   ```
   Step N: description of the step and any warnings that could happen if ran
   Command: `<command>`
   ```

3. Error Response:
   ```
   Error Analysis: description of what went wrong and suggestion for how to fix
   Command: `<modified command>`
   ```
</response_format>

<important>
- Never execute destructive commands without explicit warning
- Always validate paths and resources exist before operations
- In dry-run mode, prefix explanations with "SIMULATION: "
- Track successful commands for final script generation
- Always using use markdown for your response
- Stick to the above format. Do not include any additional text such as asking the user to proceed
</important>'''

In [ ]:
#| exports
chat = Chat(model, sp=action_sp)
bba = chat.toolloop

## Main 

In [ ]:
#| exports
def get_history(n):
    try: return subprocess.check_output(['tmux', 'capture-pane', '-p', '-S', f'-{n}'], text=True)
    except subprocess.CalledProcessError: return None

In [ ]:
#| exports
def run_cmd(
    desc:str, # description of 
    cmd:str,  # the command to run
    ):
    "Bash command to be ran with the description of why it will be ran and what it will do"
    
    print(f"\nStep: {desc}")
    print(f"Command: `{cmd}`")
    if input("Proceed? (y/n): ").lower() == 'y':
        return subprocess.run(cmd, shell=True, text=True, capture_output=True)

In [ ]:
#| export

In [ ]:
#| exports
@call_parse
def main(
    query: Param('The query to send to the LLM', str, nargs='+'),
    action: bool = False, # Run BashBuddy in action mode
    NH: bool = False, # Don't include terminal history
    n: int = 200, # Number of history lines
    code_theme: str = 'monokai', # The code theme to use when rendering BashBuddy's responses
    code_lexer: str = 'python', # The lexer to use for inline code markdown blocks
):
    md = partial(Markdown, code_theme=code_theme, inline_code_lexer=code_lexer, inline_code_theme=code_theme)
    query = ' '.join(query)
    ctxt = ''
    # Get tmux history if requested and available
    if not NH:
        history = get_history(n)
        if history: ctxt += f'<terminal_history>\n{history}\n</terminal_history>'

    # Read from stdin if available
    if not sys.stdin.isatty(): ctxt += f'\n<context>\n{sys.stdin.read()}</context>'
    
    query = f'{ctxt}\n<query>\n{query}\n</query>'
    if action:
        print(md(contents(chat(query))))
        chat.tools = [run_cmd]
        print(md(contents(bba('proceed'))))
    else: print(contents(bb(query)))

## -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()